In [1]:
# from quchem.Hamiltonian_Generator_Functions import *
# from quchem.Graph import *
# ### HAMILTONIAN start
# Molecule = 'H2'
# geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
# basis = 'sto-3g'


# ### Get Hamiltonian
# Hamilt = Hamiltonian_PySCF(Molecule,
#                      run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
#                      basis=basis,
#                      multiplicity=1,
#                      geometry=geometry)  # normally None!
# QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
# ### HAMILTONIAN end

# #####################################

# print(QubitHamiltonian)

# const_list=[]
# P_list=[]
# for Pauli, Const in QubitHamiltonian.terms.items():
#     P_list.append(Pauli)
#     const_list.append(Const)
    
# print(P_list)
# print('')
# print(const_list)
# fci_energy = Hamilt.molecule.fci_energy
# print(fci_energy)

In [2]:
fci_energy = -1.137283834488502

P_list = [(),
 ((0, 'Z'),),
 ((0, 'Z'), (1, 'Z')),
 ((2, 'Z'),),
 ((1, 'Z'), (2, 'Z'), (3, 'Z')),
 ((1, 'Z'),),
 ((0, 'Y'), (1, 'Z'), (2, 'Y'), (3, 'Z')),
 ((0, 'X'), (1, 'Z'), (2, 'X')),
 ((0, 'X'), (1, 'Z'), (2, 'X'), (3, 'Z')),
 ((0, 'Y'), (1, 'Z'), (2, 'Y')),
 ((0, 'Z'), (2, 'Z')),
 ((0, 'Z'), (1, 'Z'), (2, 'Z'), (3, 'Z')),
 ((0, 'Z'), (1, 'Z'), (2, 'Z')),
 ((0, 'Z'), (2, 'Z'), (3, 'Z')),
 ((1, 'Z'), (3, 'Z'))]

const_list = [(-0.09706626816762878+0j),
 (0.1714128264477689+0j),
 (0.17141282644776892+0j),
 (-0.22343153690813564+0j),
 (-0.22343153690813558+0j),
 (0.16868898170361207+0j),
 (0.045302615503799264+0j),
 (0.045302615503799264+0j),
 (0.045302615503799264+0j),
 (0.045302615503799264+0j),
 (0.12062523483390414+0j),
 (0.1659278503377034+0j),
 (0.1659278503377034+0j),
 (0.12062523483390414+0j),
 (0.1744128761226159+0j)]

from openfermion.ops import QubitOperator
QubitHamiltonian = QubitOperator()

for P_word, const in zip(P_list, const_list):
    QubitHamiltonian+=QubitOperator(P_word, const)
QubitHamiltonian

(-0.09706626816762878+0j) [] +
(0.045302615503799264+0j) [X0 Z1 X2] +
(0.045302615503799264+0j) [X0 Z1 X2 Z3] +
(0.045302615503799264+0j) [Y0 Z1 Y2] +
(0.045302615503799264+0j) [Y0 Z1 Y2 Z3] +
(0.1714128264477689+0j) [Z0] +
(0.17141282644776892+0j) [Z0 Z1] +
(0.1659278503377034+0j) [Z0 Z1 Z2] +
(0.1659278503377034+0j) [Z0 Z1 Z2 Z3] +
(0.12062523483390414+0j) [Z0 Z2] +
(0.12062523483390414+0j) [Z0 Z2 Z3] +
(0.16868898170361207+0j) [Z1] +
(-0.22343153690813558+0j) [Z1 Z2 Z3] +
(0.1744128761226159+0j) [Z1 Z3] +
(-0.22343153690813564+0j) [Z2]

In [3]:
const_list

[(-0.09706626816762878+0j),
 (0.1714128264477689+0j),
 (0.17141282644776892+0j),
 (-0.22343153690813564+0j),
 (-0.22343153690813558+0j),
 (0.16868898170361207+0j),
 (0.045302615503799264+0j),
 (0.045302615503799264+0j),
 (0.045302615503799264+0j),
 (0.045302615503799264+0j),
 (0.12062523483390414+0j),
 (0.1659278503377034+0j),
 (0.1659278503377034+0j),
 (0.12062523483390414+0j),
 (0.1744128761226159+0j)]

In [4]:
from quchem.Ansatz_Generator_Functions import *
# n_electrons=Hamilt.molecule.n_electrons
# n_qubits= Hamilt.molecule.n_qubit
n_electrons=2
n_qubits= 4

ansatz_obj = Ansatz(n_electrons, n_qubits)



Sec_Quant_CC_ops_ia, Sec_Quant_CC_ops_ijab, theta_parameters_ia, theta_parameters_ijab=ansatz_obj.Get_ia_and_ijab_terms(single_cc_amplitudes=None, 
                                                                                                             double_cc_amplitudes=None, 
                                                                                                             singles_hamiltonian=None,
                                                                                                             doubles_hamiltonian=None, 
                                                                                                             tol_filter_small_terms = None)

Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(Sec_Quant_CC_ops_ia,
                                                                                                                    Sec_Quant_CC_ops_ijab,
                                                                                                                   transformation='BK')


In [5]:
input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis()
print(input_state)

UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(input_state,
                                     Qubit_Op_list_Second_Quant_CC_Ops_ia, 
                                     Qubit_Op_list_Second_Quant_CC_Ops_ijab)

UCCSD_ansatz_Q_Circ =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(Theta_param_list_ia=theta_parameters_ia, 
                                         Theta_param_list_ijab=theta_parameters_ijab,
                                         ia_first=True)
UCCSD_ansatz_Q_Circ

[1. 0. 0. 0.]


┌─────────┐   ┌──────────┐              ┌──────────┐
0: ───X───H──────────@───────────────────────@───H───────────Rx(0.5π)───@───────────────────────@───────────Rx(-0.5π)──────────────@───────────────────────@──────────H────────────────────@───────────────────@───────────H─────────────H───────────@───────────────────────@───────────H───Rx(0.5π)───@──────────────@───Rx(-0.5π)───Rx(0.5π)───@──────────────────────@───Rx(-0.5π)───Rx(0.5π)───@──────────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@──────────────────────@───Rx(-0.5π)───H──────────@───────────────────────────────@───────────H──────────H───@───────────────────────@───H───────────
                     │                       │                          │                       │                                  │                       │                               │                   │                                     │                       │                          │              │                          │                      │                          │                              │                          │                      │                          │                               │                          │                       │
1: ───I───Rx(0.5π)───X───@───────────────@───X───Rx(-0.5π)───Rx(0.5π)───X───@───────────────@───X───────────Rx(-0.5π)───Rx(0.5π)───X───@───────────────@───X──────────Rx(-0.5π)────Rx(0.5π)┼─────@─────────────┼───────────@─────────────Rx(-0.5π)───X───@───────────────@───X──────────────────────────┼──────────────┼──────────────────────────X───@──────────────@───X──────────────────────────X───@──────────────────────@───X──────────────────────────┼──────────────────────┼──────────────────────────X───@───────────────────────@───X──────────────────────────┼───────────────────────┼───────────────
                         │               │                                  │               │                                          │               │                                   │     │             │           │                             │               │                              │              │                              │              │                                  │                      │                              │                      │                              │                       │                              │                       │
2: ───I───H──────────────X───Rz(-0.0π)───X───H───Rx(0.5π)───────────────────X───Rz(-0.0π)───X───Rx(-0.5π)──────────────────────────────X───Rz(-0.0π)───X───Rx(0.5π)────────────────────────X─────┼Rz(-0.0π)────X───────────┼Rx(-0.5π)────Rx(0.5π)────────X───Rz(-0.0π)───X───Rx(-0.5π)───H──────────────X───Rz(0.0π)───X───H───────────H──────────────X───Rz(0.0π)───X───H───H──────────────────────────X───@──────────────@───X───H───H──────────────────────X───@──────────────@───X───H───────────Rx(0.5π)───────X───@───────────────@───X───Rx(-0.5π)───Rx(0.5π)───────X───@───────────────@───X───Rx(-0.5π)───
                                                                                                                                                                                                 │                         │                                                                                                                                                                                │              │                                      │              │                                      │               │                                      │               │
3: ───I──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────────────Rz(0.0π)────X────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───Rz(0.0π)───X────────

In [6]:
Qubit_Op_list_Second_Quant_CC_Ops_ijab

[0.125j [X0 Z1 Y2] +
 0.125j [X0 Z1 Y2 Z3] +
 0.125j [X0 Y2] +
 0.125j [X0 Y2 Z3] +
 -0.125j [Y0 Z1 X2] +
 -0.125j [Y0 Z1 X2 Z3] +
 -0.125j [Y0 X2] +
 -0.125j [Y0 X2 Z3]]

From Helgaker, T., P. Jorgensen,and J. Olsen (2014),Molecularelectronic-structure theory(John Wiley & Sons)
we known

$$H_{2}^{ground} = 0.9939| 1100\rangle - 0.1106| 0011\rangle$$

From group theory we know only double excitation terms are important!
We can see this from our ijab operators that qubits 1 and 3 are only acted on by $Z$ therefore experience no population changes... Can even see this from the answer if we write it in BK basis:

$$H_{2}^{BK} = 0.9939| 1000\rangle - 0.1106| 0010\rangle$$

See how qubits 1 and 3 always remain at 0!

therefore can **remove** these measurements from Hamiltonian... as we know what expectation values they should give!

In the case for this Hamiltonian only $I$ and $Z$ act on these qubits! So even easier to remove (expectation values = +1 (as act on $| 0\rangle$ state!), so just remove and add to relivent H terms!

LOOK at:

PHYS. REV. X, **8**, 031022 (2018)

In [7]:
print('double e- operators:')
print(Qubit_Op_list_Second_Quant_CC_Ops_ijab)

ansatz_obj = BK_Qubit_Reduction(QubitHamiltonian,
                             n_electrons,
                             n_qubits)

indices_to_Remove = ansatz_obj.Find_Qubits_only_acted_on_by_I_or_Z(Qubit_Op_list_Second_Quant_CC_Ops_ijab)
print('')
print('qubit indices:', indices_to_Remove, 'only acted on by I or Z')

double e- operators:
[0.125j [X0 Z1 Y2] +
0.125j [X0 Z1 Y2 Z3] +
0.125j [X0 Y2] +
0.125j [X0 Y2 Z3] +
-0.125j [Y0 Z1 X2] +
-0.125j [Y0 Z1 X2 Z3] +
-0.125j [Y0 X2] +
-0.125j [Y0 X2 Z3]]

qubit indices: [1 3] only acted on by I or Z


In [8]:
list(QubitHamiltonian)

[(-0.09706626816762878+0j) [],
 (0.1714128264477689+0j) [Z0],
 (0.17141282644776892+0j) [Z0 Z1],
 (-0.22343153690813564+0j) [Z2],
 (-0.22343153690813558+0j) [Z1 Z2 Z3],
 (0.16868898170361207+0j) [Z1],
 (0.045302615503799264+0j) [Y0 Z1 Y2 Z3],
 (0.045302615503799264+0j) [X0 Z1 X2],
 (0.045302615503799264+0j) [X0 Z1 X2 Z3],
 (0.045302615503799264+0j) [Y0 Z1 Y2],
 (0.12062523483390414+0j) [Z0 Z2],
 (0.1659278503377034+0j) [Z0 Z1 Z2 Z3],
 (0.1659278503377034+0j) [Z0 Z1 Z2],
 (0.12062523483390414+0j) [Z0 Z2 Z3],
 (0.1744128761226159+0j) [Z1 Z3]]

### Can remove these terms from Hamiltonian

In [9]:
reduced_Qubit_Hamiltonian = ansatz_obj.Remove_indices_from_Hamiltonian(indices_to_Remove)
reduced_Qubit_Hamiltonian

(0.2460355896585992+0j) [] +
(0.09060523100759853+0j) [X0 X2] +
(0.09060523100759853+0j) [Y0 Y2] +
(0.3428256528955378+0j) [Z0] +
(0.5731061703432151+0j) [Z0 Z2] +
(-0.4468630738162712+0j) [Z2]

### Can remove these terms from Anstz

In [10]:
reduced_ijab_CC = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ijab,
                                                             indices_to_Remove)
reduced_ijab_CC

[0.5j [X0 Y2] +
 -0.5j [Y0 X2]]

# NEXT need to re-label everything

In [11]:
relabel_dict, relabelled_reduced_Qubit_Hamiltonian = ansatz_obj.Re_label_Hamiltonian(reduced_Qubit_Hamiltonian)

print('qubit relabelling dict = ', relabel_dict)

relabelled_reduced_Qubit_Hamiltonian

qubit relabelling dict =  {0: 0, 2: 1}


(0.2460355896585992+0j) [] +
(0.09060523100759853+0j) [X0 X1] +
(0.09060523100759853+0j) [Y0 Y1] +
(0.3428256528955378+0j) [Z0] +
(0.5731061703432151+0j) [Z0 Z1] +
(-0.4468630738162712+0j) [Z1]

In [12]:
relabelled_reduced_ijab_CC = ansatz_obj.Re_label_CC_qubit_operators(relabel_dict, reduced_ijab_CC)
relabelled_reduced_ijab_CC

[0.5j [X0 Y1] +
 -0.5j [Y0 X1]]

## Find New input state

In [13]:
print('old input = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())
print('BUT following indices removed:', indices_to_Remove)
ansatz_obj.New_BK_HF_state(indices_to_Remove)

old input =  [1. 0. 0. 0.]
BUT following indices removed: [1 3]


array([1., 0.])

# Find what new FCI energy is
- here should be the same as true answer!

In [14]:
from openfermion import qubit_operator_sparse
new_Molecular_H_MATRIX =  qubit_operator_sparse(relabelled_reduced_Qubit_Hamiltonian)

from scipy.linalg import eig
eig_values, eig_vectors = eig(new_Molecular_H_MATRIX.todense())
new_FCI_Energy = min(eig_values)


print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', fci_energy)

new_FCI =  (-1.1372838344885006+0j) VS old FCI: -1.137283834488502


In [15]:
from quchem.LCU_method import *

In [16]:
# Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(relabelled_reduced_Qubit_Hamiltonian)

# commutativity_flag = 'AC' ## <- defines relationship between sets!!!
# plot_graph = False
# Graph_colouring_strategy='largest_first'
# anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
# anti_commuting_sets

# Graph_sets_dict={}
# for key in anti_commuting_sets:
#     terms=[]
#     for QubitOp in anti_commuting_sets[key]:
#         terms.append(*tuple(QubitOp.terms.items()))
#     Graph_sets_dict[key]=(terms)
# Graph_sets_dict

In [17]:
Graph_sets_dict = {0: [(((0, 'Z'), (1, 'Z')), (0.5731061703432151+0j))],
 1: [((), (0.2460355896585992+0j))],
 2: [(((0, 'Y'), (1, 'Y')), (0.09060523100759853+0j)),
  (((0, 'Z'),), (0.3428256528955378+0j))],
 3: [(((1, 'Z'),), (-0.4468630738162712+0j)),
  (((0, 'X'), (1, 'X')), (0.09060523100759853+0j))]}


anti_commuting_sets={}
for key in Graph_sets_dict:
    terms=[]
    for PauliWord, const in Graph_sets_dict[key]:
        terms.append(QubitOperator(PauliWord, const))
    anti_commuting_sets[key]=terms
anti_commuting_sets

{0: [(0.5731061703432151+0j) [Z0 Z1]],
 1: [(0.2460355896585992+0j) []],
 2: [(0.09060523100759853+0j) [Y0 Y1], (0.3428256528955378+0j) [Z0]],
 3: [(-0.4468630738162712+0j) [Z1], (0.09060523100759853+0j) [X0 X1]]}

In [18]:
from quchem.latex_tables import *

In [19]:
latex_table_Hamiltonian(anti_commuting_sets, 'Latex_table1')
latex_table_seq_rot(anti_commuting_sets, 'Latex_table2', 0)
latex_table_LCU(anti_commuting_sets, 'Latex_table3', 0)
latex_table_LCU_R_op(anti_commuting_sets, 'Latex_table4', 0)

\begin{tabular}{rl}
\toprule
 l index &                                                        \$H\_\{S\_\{l\}\}\$ \\
\midrule
       0 &                                  [(0.5731061703432151+0j) [Z0 Z1]] \\
       1 &                                       [(0.2460355896585992+0j) []] \\
       2 &   [(0.09060523100759853+0j) [Y0 Y1], (0.3428256528955378+0j) [Z0]] \\
       3 &  [(-0.4468630738162712+0j) [Z1], (0.09060523100759853+0j) [X0 X1]] \\
\bottomrule
\end{tabular}

\begin{tabular}{rrllll}
\toprule
 l index &  \$\textbackslash gamma\_\{l\}\$ &                                    \$rac\{H\_\{S\_\{l\}\}\}\{\textbackslash gamma\_\{l\}\}\$ &         Pn & \$\textbackslash mathcal\{X\_\{nk\}\}\$ &        \$\textbackslash theta\_\{nk\}\}\$ \\
\midrule
       2 &      0.354597 &  [(0.2555163685543363+0j) [Y0 Y1], (0.9668047297157811+0j) [Z0]] &  1 [Y0 Y1] &  [(-1+0j) [X0 Y1]] &  [1.3124145631682165] \\
       3 &      0.455956 &  [(-0.9800573522118143+0j) [Z1], (0.198714836827973+0j) [X